In [2]:
#import dependencies
import keras
from tensorflow.keras.models import Sequential
from keras.utils import np_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np
import cv2 as cv
from matplotlib import pyplot

In [3]:
# download and split the data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print("training data = ", x_train.shape)
print("testing data = ", x_test.shape)

training data =  (50000, 32, 32, 3)
testing data =  (10000, 32, 32, 3)


In [4]:
# Normalize the data to speed up training
mean = np.mean(x_train)
std = np.std(x_train)
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)


In [5]:
mean, std

(120.70748, 64.150024)

In [6]:
def buildModel():
    # build the model
    
    # number of hidden units variable 
    # we are declaring this variable here and use it in our CONV layers to make it easier to update from one place
    base_hidden_units = 64 # changed from 32 to 64
    
    # l2 regularization hyperparameter
    weight_decay = 1e-4 
    num_classes = 10
    
    # instantiate an empty sequential model 
    model = Sequential()
    
    # CONV1
    # notice that we defined the input_shape here because this is the first CONV layer. 
    # we don’t need to do that for the remaining layers
    model.add(Conv2D(base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(32, 32, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    
    # CONV2
    model.add(Conv2D(base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    # CONV3
    model.add(Conv2D(2*base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    
    # CONV4
    model.add(Conv2D(2*base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))
    
    # CONV5
    model.add(Conv2D(4*base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    
    # CONV6
    model.add(Conv2D(4*base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))
    
    # FC7
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    
    # print model summary
    #model.summary()
    model.load_weights('model.125epochs.hln64bs256.hdf5')

    return model


cnnModel = buildModel()

In [7]:
def createPyramid(image, levels):
    '''
    creates an image pyramid for given number of levels
    '''
    gaussPyr = [image]
    
    for i in range(levels-1):
        image = cv.pyrDown(image)
        gaussPyr.append(image)
        
    return gaussPyr

In [8]:
labelNames = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [10]:
imageFiles = ['carw.png', 'deerw.png', 'deer.jpg', 'frogw.png', 'planew.png', 'shipw.png']
for imgName in imageFiles:
  # reading the image
  img = cv.imread(imgName)
  print("Shape of the original imgae: {}".format(img.shape))
  img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
  # resize the image to 256x256
  imgResized = cv.resize(img, (256,256), interpolation = cv.INTER_AREA)
  print("Shape of the resized image: {}".format(imgResized.shape))
  pyramid = createPyramid(imgResized, 4) # calling 4 levels
  print("Shape of the lowest pyramid image: {}".format(pyramid[3].shape))
  # adding a new axis on pyramid[3]
  evalImage = pyramid[3][np.newaxis,:,:,:]
  cv.imwrite(imgName.split('.')[0]+'pyr.jpg', evalImage)
  print("Shape of the image being evaluated {}".format(evalImage.shape))
  evalImage = (evalImage-mean)/(std+1e-7)

  # prediction
  pred = cnnModel.predict(evalImage)
  np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
  print("The probablities: {}".format(pred))
  print("Predicted class: {}\n".format(labelNames[pred.argmax()]))

Shape of the original imgae: (720, 1280, 3)
Shape of the resized image: (256, 256, 3)
Shape of the lowest pyramid image: (32, 32, 3)
Shape of the image being evaluated (1, 32, 32, 3)
The probablities: [[0.00 0.92 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.07]]
Predicted class: automobile

Shape of the original imgae: (720, 1280, 3)
Shape of the resized image: (256, 256, 3)
Shape of the lowest pyramid image: (32, 32, 3)
Shape of the image being evaluated (1, 32, 32, 3)
The probablities: [[0.01 0.05 0.00 0.01 0.00 0.00 0.00 0.00 0.00 0.94]]
Predicted class: truck

Shape of the original imgae: (330, 750, 3)
Shape of the resized image: (256, 256, 3)
Shape of the lowest pyramid image: (32, 32, 3)
Shape of the image being evaluated (1, 32, 32, 3)
The probablities: [[0.00 0.00 0.02 0.13 0.77 0.04 0.00 0.03 0.00 0.00]]
Predicted class: deer

Shape of the original imgae: (720, 1280, 3)
Shape of the resized image: (256, 256, 3)
Shape of the lowest pyramid image: (32, 32, 3)
Shape of the image being ev